In [ ]:
import os
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

from metrics import print_overall_performance_mean_std

In [ ]:
dropout = 0.5
alpha = 1
seeds = [10, 20, 30, 40, 50]
upset_choices = ['upset', 'cycle inconsistency']
MSE_choices = ['MSE']
selected_metrics = MSE_choices + upset_choices
METRICS_NUM = len(selected_metrics)
baselines = ['spectral', 'row_norm_spectral', 'trivial']
NUM_BASELINES = len(baselines)
NUM_UPSET_CHOICES = len(upset_choices)
NUM_MSE_CHOICES = len(MSE_choices)
GNN_selection_choices = ['lr', 'train with', 'upset coeff', 'spectral coeff', 'row norm spectral coeff', 'cycle coeff', 'baseline']
GNN_selection_choices_curr = ['train with', 'upset coeff', 'spectral coeff', 'row norm spectral coeff', 'cycle coeff', 'baseline']
GNN_CHOICES_NUM = len(GNN_selection_choices)
GNN_CHOICES_NUM_CURR = len(GNN_selection_choices_curr)
all_GNNs = ['GNNSync']
train_with_list = ['innerproduct']

GNN_variant_names = train_with_list
GNN_NUM = 1

def generate_method_str_and_compare_names_all(all_methods=baselines):
    method_str = ''
    for method_name in all_methods:
        method_str += method_name
    compare_names_all = []
    for method_name in all_methods:
        if method_name not in ['DIGRAC', 'ib', 'DIGRAC_Att']:
            compare_names_all.append(method_name)
        else:
            for GNN_type in GNN_variant_names:
                compare_names_all.append(method_name+'_'+GNN_type)
    return method_str, compare_names_all

'''
methods_of_interest = ['0001', '1000', '1001'] + baselines
methods_of_interest_print = ['GNNSync-cycle', 'GNNSync-upset', 'GNNSync-sum', 'Spectral', 'Spectral_RN', 'Trivial']


# below for linear comb ablation study
baselines = ['trivial']
methods_of_interest = ['0.0', '0.1', '0.3', '0.5', '0.7', '0.9', '1000'] + baselines
methods_of_interest_print = [r'$\tau=0.0$ (GNNSync-cycle)', r'$\tau=0.1$', r'$\tau=0.3$', r'$\tau=0.5$', r'$\tau=0.7$', r'$\tau=0.9$', 'GNNSync-upset', 'Trivial']
'''
# below for ablation study
baselines = ['trivial']
methods_of_interest = ['0001', '0001g', '00012', '0001h', '0001t', '0001s'] + baselines
methods_of_interest_print = ['Default', 'No Projected Gradient Steps', 'Not End-to-End', r'Separate $\{\mathbf{H}^{(l)}\}$', r'Trainable $\{\alpha_\gamma\}$', '"Spectral" for Input Features', 'Trivial']


innerproduct_ind = [0]
INNERPRODUCT_GNN_NUM = len(innerproduct_ind)
innerproduct_bool = np.zeros(GNN_NUM, dtype=bool)
for i in innerproduct_ind:
    innerproduct_bool[i] = True

GNN_names_innerproduct = ['DIGRAC_innerproduct']


compare_names_all = methods_of_interest
METHODS_NUM = len(compare_names_all)

In [ ]:
p_list_dict = {}
eta_list_dict = {}
outlier_style_list_dict = {}
N_list_dict = {}
for dataset_name in ['ERO', 'BAO', 'RGGO']:
    N_list_dict[dataset_name] = [360]
    p_list_dict[dataset_name] = [0.05, 0.1, 0.15]
    eta_list_dict[dataset_name] = {}
    eta_list_dict[dataset_name][0.05] = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    eta_list_dict[dataset_name][0.1] = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    eta_list_dict[dataset_name][0.15] = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    outlier_style_list_dict[dataset_name] = ['multi_normal0', 'multi_normal1','gamma', 'block_normal6']

In [ ]:
def generate_save_name(dataset='ERO/p5N360eta0styleuniform_bisync', all_methods=all_GNNs, train_with='dist', upset_coeff=1.0, spectral_coeff=1.0, row_norm_spectral_coeff=1.0,
                           lr=0.01, hidden=8, num_trials=10, train_ratio=1, test_ratio=1,  AllTrain=True, sync_baseline='spectral', sigma=1.0, cycle_coeff=1, trainable_alpha=False, 
                           spectral_step_num=5, two_step=False, same_H=False, no_reg=True):
    default_name_base = ''
    if 'GNNSync' in all_methods:
        default_name_base += 'dropout' + str(int(100*dropout))
        default_name_base += 'upset_coe' + str(int(100*upset_coeff)) + 'cycle_coe' + str(int(100*cycle_coeff))
        if cycle_coeff > 0 and not no_reg:
            default_name_base += 'reg_coe100'
        if spectral_step_num > 0:
            default_name_base += 'spectral_step_num' + str(spectral_step_num) + 'alpha100train_alpha' + str(trainable_alpha)
            if not same_H:
                default_name_base += 'separate_graphs'
        default_name_base += 'hid' + str(hidden) + 'lr' + str(int(1000*lr))
        default_name_base += 'use' + str(sync_baseline) + 'SGD'

    default_name_base +=  'trials' + str(num_trials)
    if dataset[:3] in ['ERO', 'BAO', 'SBM', 'RGG'] and set(seeds) != set([10, 20, 30, 40, 50]):
        default_name_base += 'seeds' + '_'.join([str(value) for value in np.array(seeds).flatten()])
    if two_step:
        default_name_base = 'two_step_' + default_name_base
    return default_name_base

In [ ]:
def dataset_name_preprocess(dataset, p=0.05, outlier_style='uniform', eta=0.1, N=360, k=2, size_ratio=1, prob_ratio=2):
    if dataset[-1]!='/':
        dataset += '/'

    if dataset[:3] in ['ERO', 'BAO', 'RGG']:
        hidden = 8
        default_name_base = 'p' + str(int(100*p)) + 'N' + str(N)
        default_name_base += 'eta' + str(int(100*eta)) + 'style' + str(outlier_style)
        if dataset[:4] == 'RGGO':
            dataset = 'RGGO/' + default_name_base
            if outlier_style == 'gamma' and N == 360:
                dataset_print = 'RGGO(k={}, p={}, $\eta$={})'.format(k, p, eta)
            else:
                dataset_print = 'RGGO(k={}, p={}, style={},$\eta$={})'.format(k, p, outlier_style, eta)   
        else:
            dataset = dataset[:3] + '/' + default_name_base
            if outlier_style == 'gamma' and N == 360:
                dataset_print = dataset[:3] + '(k={}, p={}, $\eta$={})'.format(k, p, eta)
            else:
                dataset_print = dataset[:3] + '(k={}, p={}, style={},$\eta$={})'.format(k, p, outlier_style, eta)
    dataset += 'k' + str(k)
    dataset_print = str(k) + '&' + str(p) + '&' + str(eta) # updated for new table generation
    
    return dataset_print, dataset, hidden 

In [ ]:
lr_list = [0.005]
sync_baseline_list = ['row_norm_spectral', 'spectral']
upset_coeff_list = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
spectral_coeff_list = [0]
row_norm_spectral_coeff_list = [0]
cycle_coeff_list = [0, 1]
innerproduct_ind_correspondence_dict = {}

i = 0
for lr_ind, lr in enumerate(lr_list):
    for train_ind, train_with in enumerate(train_with_list):
        for upset_coeff_ind, upset_coeff in enumerate(upset_coeff_list):
            for spectral_coeff_ind, spectral_coeff in enumerate(spectral_coeff_list):
                for row_norm_spectral_coeff_ind, row_norm_spectral_coeff in enumerate(row_norm_spectral_coeff_list):
                    for cycle_coeff_ind, cycle_coeff in enumerate(cycle_coeff_list):
                        for base_ind, sync_baseline in enumerate(sync_baseline_list):
                            for method_ind in range(INNERPRODUCT_GNN_NUM):
                                innerproduct_ind_correspondence_dict[i] = [lr_ind, train_ind, upset_coeff_ind, spectral_coeff_ind, \
                                                                row_norm_spectral_coeff_ind, cycle_coeff_ind, base_ind, method_ind]
                                i += 1
                            
innerproduct_cases_num = len(innerproduct_ind_correspondence_dict.keys())
print(innerproduct_cases_num)

In [ ]:
def GNN_load_results(dataset, all_methods=all_GNNs, train_with='innerproduct', 
                                   upset_coeff=1, spectral_coeff=1, row_norm_spectral_coeff=1, lr=0.01, hidden=32, num_trials=10, 
                                   train_ratio=0.8, test_ratio=0.1,  AllTrain=True, sync_baseline='spectral', 
                                   sigma=1, cycle_coeff=1, trainable_alpha=False, spectral_step_num=5, two_step=False, same_H=False, no_reg=True):
    save_name = generate_save_name(dataset=dataset, all_methods=all_GNNs, train_with=train_with, 
                                   upset_coeff=upset_coeff, spectral_coeff=spectral_coeff, row_norm_spectral_coeff=row_norm_spectral_coeff,
                                   lr=lr, hidden=hidden, num_trials=num_trials, train_ratio=train_ratio, test_ratio=test_ratio, 
                                    AllTrain=AllTrain, sync_baseline=sync_baseline, sigma=sigma, cycle_coeff=cycle_coeff, trainable_alpha=trainable_alpha, 
                                    spectral_step_num=spectral_step_num, two_step=two_step, same_H=same_H, no_reg=no_reg)
    method_str, _ = generate_method_str_and_compare_names_all(all_methods)
    dir_name = '../result_arrays/'+dataset
    try:
        mse_res = np.load(os.path.join(dir_name,'MSE',method_str,save_name) + '.npy')[:, :, 2]
        final_upset = np.load(os.path.join(dir_name,'upset',method_str,save_name) + '.npy')
    except FileNotFoundError:
        print('File not found: ', os.path.join(dir_name,'MSE',method_str,save_name))
    return mse_res, final_upset

In [ ]:
def GNN_selection(dataset, train_ratio, test_ratio, AllTrain, hidden, num_trials, constraint=None):
    pretrain_epochs = 50
    sigma = 1
    full_results_innerproduct = 1000*np.ones((METRICS_NUM, INNERPRODUCT_GNN_NUM * innerproduct_cases_num))
    final_ind_innerproduct = 0
    trainable_alpha = False
    has_result = False
    two_step = False
    same_H = True
    no_reg = True
    spectral_step_num = 5
    for lr_ind, lr in enumerate(lr_list):
        for train_ind, train_with in enumerate(train_with_list):
            for upset_coeff_ind, upset_coeff in enumerate(upset_coeff_list):
                for spectral_coeff_ind, spectral_coeff in enumerate(spectral_coeff_list):
                    for row_norm_spectral_coeff_ind, row_norm_spectral_coeff in enumerate(row_norm_spectral_coeff_list):
                        for cycle_coeff_ind, cycle_coeff in enumerate(cycle_coeff_list):
                            for base_ind, sync_baseline in enumerate(sync_baseline_list):
                                try:
                                    if constraint == 'row_norm_spectral':
                                        assert upset_coeff == 1 and spectral_coeff == 0 and row_norm_spectral_coeff == 0 and cycle_coeff == 0 and sync_baseline == 'row_norm_spectral'
                                    elif constraint == 'spectral':
                                        assert upset_coeff == 1 and spectral_coeff == 0 and row_norm_spectral_coeff == 0 and cycle_coeff == 0 and sync_baseline == 'spectral'
                                    elif constraint == 'trainable_alpha':
                                        assert upset_coeff == 1 and spectral_coeff == 0 and row_norm_spectral_coeff == 0 and cycle_coeff == 0 and sync_baseline == 'row_norm_spectral'
                                        trainable_alpha = True
                                    elif constraint == 'no_GPM_steps':
                                        assert upset_coeff == 0 and spectral_coeff == 0 and row_norm_spectral_coeff == 0 and cycle_coeff == 1 and sync_baseline == 'row_norm_spectral'
                                        spectral_step_num = 0
                                    elif constraint == 'two_step':
                                        assert upset_coeff == 0 and spectral_coeff == 0 and row_norm_spectral_coeff == 0 and cycle_coeff == 1 and sync_baseline == 'row_norm_spectral'
                                        spectral_step_num = 0
                                        two_step = True
                                    elif constraint == 'two_step_separate_H':
                                        assert upset_coeff == 0 and spectral_coeff == 0 and row_norm_spectral_coeff == 0 and cycle_coeff == 1 and sync_baseline == 'row_norm_spectral'
                                        spectral_step_num = 0
                                        two_step = True
                                        same_H = False
                                    elif constraint == 'separate_H':
                                        assert upset_coeff == 0 and spectral_coeff == 0 and row_norm_spectral_coeff == 0 and cycle_coeff == 1 and sync_baseline == 'row_norm_spectral'
                                        same_H = False
                                    elif constraint == 'no_reg':
                                        assert sync_baseline == 'row_norm_spectral' and upset_coeff == 0 and cycle_coeff == 1.0
                                        no_reg = True
                                    if len(constraint) == 3:
                                        assert sync_baseline == 'row_norm_spectral' and upset_coeff == float(constraint) and cycle_coeff == 1.0
                                    elif len(constraint) == 4:
                                        assert sync_baseline == 'row_norm_spectral' and upset_coeff == float(constraint[0]) and spectral_coeff == float(constraint[1]) and row_norm_spectral_coeff == float(constraint[2]) and cycle_coeff == float(constraint[3])
                                    elif len(constraint) == 5:
                                        assert upset_coeff == float(constraint[0]) and spectral_coeff == float(constraint[1]) and row_norm_spectral_coeff == float(constraint[2]) and cycle_coeff == float(constraint[3])
                                        if constraint[4] == 's':
                                            assert sync_baseline == 'spectral'
                                        elif constraint[4] == 't':
                                            assert sync_baseline == 'row_norm_spectral'
                                            trainable_alpha = True
                                        elif constraint[4] == 'h':
                                            assert sync_baseline == 'row_norm_spectral'
                                            same_H = False
                                        elif constraint[4] == 'g': # no GPM steps
                                            assert sync_baseline == 'row_norm_spectral'
                                            spectral_step_num = 0
                                        elif constraint[4] == '2': # two steps
                                            assert sync_baseline == 'row_norm_spectral'
                                            spectral_step_num = 0
                                            two_step = True
                                        elif constraint[4] == 'w': # two steps and separate H
                                            assert sync_baseline == 'row_norm_spectral'
                                            spectral_step_num = 0
                                            two_step = True
                                            same_H = False
                                    
                                    # print('1', upset_coeff, cycle_coeff)
                                    mse, upsets = GNN_load_results(dataset=dataset, all_methods=all_GNNs, train_with=train_with, 
                                            upset_coeff=upset_coeff, spectral_coeff=spectral_coeff, row_norm_spectral_coeff=row_norm_spectral_coeff, 
                                            lr=lr, hidden=hidden, num_trials=num_trials, train_ratio=train_ratio, test_ratio=test_ratio, 
                                            AllTrain=AllTrain, sync_baseline=sync_baseline, sigma=sigma, cycle_coeff=cycle_coeff, 
                                            trainable_alpha=trainable_alpha, spectral_step_num=spectral_step_num, two_step=two_step, same_H=same_H, no_reg=no_reg)

                                    mean_mse = np.nanmean(mse[innerproduct_bool].swapaxes(0,-1), axis=1)
                                    full_results_innerproduct[:NUM_MSE_CHOICES, final_ind_innerproduct: final_ind_innerproduct + INNERPRODUCT_GNN_NUM] = mean_mse


                                    mean_upsets = upsets[innerproduct_bool].swapaxes(0,-1).mean(axis=1)
                                    full_results_innerproduct[NUM_MSE_CHOICES:, final_ind_innerproduct: final_ind_innerproduct + INNERPRODUCT_GNN_NUM] = mean_upsets

                                    has_result = True
                                except FileNotFoundError:
                                    pass
                                except AssertionError:
                                    pass
                                except ValueError:
                                    pass
                                final_ind_innerproduct += INNERPRODUCT_GNN_NUM
    if has_result:
        best_ind = np.zeros((METRICS_NUM, 1))
        best_vals = np.ones((METRICS_NUM, 1))
        best_vals[:] = np.nan
        full_results_innerproduct = np.nan_to_num(full_results_innerproduct, nan=1000)
        if dataset[:3] in ['ERO', 'BAO', 'SBM', 'RGG']:
            for mse_ind in range(NUM_MSE_CHOICES):
                best_ind[mse_ind, 0] = full_results_innerproduct[mse_ind].argmin()
                best_vals[mse_ind, 0] = np.nanmin(full_results_innerproduct[mse_ind])
        

        for upset_ind in range(NUM_UPSET_CHOICES):
            best_vals[NUM_MSE_CHOICES+upset_ind, 0] = np.nanmin(full_results_innerproduct[NUM_MSE_CHOICES+upset_ind])
            best_ind[NUM_MSE_CHOICES+upset_ind, 0] = full_results_innerproduct[NUM_MSE_CHOICES+upset_ind].argmin()
        selected_indices = np.zeros((METRICS_NUM, 1, GNN_CHOICES_NUM+1))
        mse_res = np.zeros((METRICS_NUM, 1, 10, NUM_MSE_CHOICES))
        mse_res[:] = np.nan
        final_upset = np.zeros((METRICS_NUM, 1, 10, NUM_UPSET_CHOICES)) # the first "2" means dist and innerproduct
        final_upset[:] = np.nan
        for i in range(METRICS_NUM):
            j = 0
            selected_indices[i, j] = innerproduct_ind_correspondence_dict[best_ind[i, j]]
                
            lr = lr_list[int(selected_indices[i, j, 0])]
            train_with = train_with_list[int(selected_indices[i, j, 1])]
            upset_coeff = upset_coeff_list[int(selected_indices[i, j, 2])]
            spectral_coeff = spectral_coeff_list[int(selected_indices[i, j, 3])]
            row_norm_spectral_coeff = row_norm_spectral_coeff_list[int(selected_indices[i, j, 4])]
            cycle_coeff = cycle_coeff_list[int(selected_indices[i, j, 5])]
            sync_baseline = sync_baseline_list[int(selected_indices[i, j, 6])]

            sel_ind = int(selected_indices[i, j, -1])
            GNN_selected = GNN_names_innerproduct[sel_ind]
            selected_vals = [lr, train_with, upset_coeff, spectral_coeff,\
                            row_norm_spectral_coeff, sync_baseline, pretrain_epochs]
            print_str = GNN_selected + ' on ' + dataset + ': '
            for k in range(GNN_CHOICES_NUM):
                print_str += GNN_selection_choices[k] + '=' + str(selected_vals[k]) + ', '
            print_str += 'for the best ' + selected_metrics[i]
            mse, upsets = GNN_load_results(dataset=dataset, all_methods=all_GNNs, train_with=train_with, 
                                        upset_coeff=upset_coeff, spectral_coeff=spectral_coeff, row_norm_spectral_coeff=row_norm_spectral_coeff, 
                                        lr=lr, hidden=hidden, num_trials=num_trials, train_ratio=train_ratio, test_ratio=test_ratio, 
                                        AllTrain=AllTrain, sync_baseline=sync_baseline, sigma=sigma, cycle_coeff=cycle_coeff, 
                                        trainable_alpha=trainable_alpha, spectral_step_num=spectral_step_num, two_step=two_step, same_H=same_H, no_reg=no_reg)

            mse_res[i, j] = (mse[innerproduct_bool])[sel_ind]
            upsets_res = (upsets[innerproduct_bool])[sel_ind]
            final_upset[i, j] = np.array(upsets_res)
        return mse_res, final_upset, selected_indices
    else:
        raise FileNotFoundError

In [ ]:
def extract_results(dataset, upset_coeff=1.0, p=0.1, 
                       AllTrain=True, eta=0.1, lr=0.05, hidden=8, 
                        N=350, outlier_style='gamma', train_ratio = 0.8, test_ratio = 0.1, dropout=0.5, sigma=1.0, k=2,
                           methods_of_interest=methods_of_interest):
    num_trials = 2
    alpha = 1
    seeds = [10, 20, 30, 40, 50]
    
    dataset_print, dataset, hidden = dataset_name_preprocess(dataset, p, outlier_style, eta, N, k)

    if dataset[:3] not in ['ERO', 'BAO', 'RGG']:
        num_trials = 10
        AllTrain = True
        train_ratio = 1
        test_ratio = 1
        seeds = [10]
    
    final_upset_full_list_dist = []
    mse_res_full_list_innerproduct = []
    final_upset_full_list_innerproduct = []
    for ind, constraint in enumerate(methods_of_interest):
        if ind < len(methods_of_interest)-len(baselines):
            mse_res_full, final_upset_full, selected_indices = GNN_selection(dataset, train_ratio, test_ratio, AllTrain, hidden, num_trials, constraint=constraint)
            mse_res_full_list_innerproduct.append(mse_res_full)
            final_upset_full_list_innerproduct.append(final_upset_full)
        


    mse_res_all = np.zeros((METRICS_NUM, METHODS_NUM, num_trials*len(seeds), NUM_MSE_CHOICES))
    mse_res_all[:] = np.nan
    final_upset_all = np.zeros((METRICS_NUM, METHODS_NUM, num_trials*len(seeds), NUM_UPSET_CHOICES))
    final_upset_all[:] = np.nan
    dir_name = '../result_arrays/'+dataset
    for i in range(METRICS_NUM):
        compare_names_all = methods_of_interest
        mse_res = mse_res_full_list_innerproduct[0][i]
        final_upset = final_upset_full_list_innerproduct[0][i]
        for j in np.arange(1, len(methods_of_interest)-len(baselines)):
            mse_res = np.concatenate((mse_res, mse_res_full_list_innerproduct[j][i]), axis=0)
            final_upset = np.concatenate((final_upset, final_upset_full_list_innerproduct[j][i]), axis=0)


        # include baseline results
        compare_names_baselines = []
        for baseline in baselines:
            save_name = generate_save_name(dataset=dataset, all_methods=[baseline], num_trials=num_trials, 
                                        train_ratio=train_ratio, test_ratio=test_ratio,  AllTrain=AllTrain)
            method_str, compare_names_baselines = generate_method_str_and_compare_names_all([baseline])
            try:
                mse_res = np.concatenate((mse_res, np.load(os.path.join(dir_name,'MSE',method_str,save_name) + '.npy')[:, :, 2]), axis=0)
                final_upset = np.concatenate((final_upset, np.load(os.path.join(dir_name,'upset',method_str,save_name) + '.npy')), axis=0)
                compare_names_baselines.append(baseline)
            except FileNotFoundError:
                print(os.path.join(dir_name,'MSE',method_str,save_name) + '.npy')
            except ValueError:
                print(os.path.join(dir_name,'MSE',method_str,save_name) + '.npy ValueError!')
        compare_names_all =  compare_names_all + compare_names_baselines

        
        mse_res_all[i] = mse_res
        final_upset_all[i] = final_upset
        
    return dataset_print, mse_res_all, final_upset_all, compare_names_all

In [ ]:
def analysis_results(dataset='ERO', outlier_style='gamma'):
    dataset_name_full = []
    mse_res_all_full = []
    final_upset_all_full = []
    for k in [2, 3, 4]:
        for p in p_list_dict[dataset]:
            for eta in eta_list_dict[dataset][p]:
                for N in N_list_dict[dataset]:
                    try:
                        dataset_long, mse_res_all, final_upset_all, _ = extract_results(dataset=dataset, 
                            p=p, eta=eta, outlier_style=outlier_style, N=N, k=k)
                        dataset_name_full.append(dataset_long)
                        mse_res_all_full.append(mse_res_all)
                        final_upset_all_full.append(final_upset_all)
                    except FileNotFoundError:
                        print('No result yet for {}, p={}, eta={}, outlier style = {}, N = {}, k={}.'.format(dataset,
                            p, eta, outlier_style, N, k))


    full_results = np.concatenate((np.array(mse_res_all_full), np.array(final_upset_all_full)), axis=-1)
    for i in range(METRICS_NUM):
        for j in range(METRICS_NUM):
            if i == 0 and j == 0:
                results_to_print = full_results[:,j,:,:,i].swapaxes(0,2)
                if not np.isnan(results_to_print).all():
                    dataset_name_print = dataset_name_full
                    compare_names_print = compare_names_all
                    title_name = selected_metrics[i] + ' with best ' + selected_metrics[j] + ' on ' + dataset + '_' + outlier_style
                    print_overall_performance_mean_std(title_name, results_to_print, 
                                    compare_names_print, dataset_name_print, True)

In [ ]:
analysis_results('BAO', 'multi_normal0')

In [ ]:

save = True
figure_markers = ['*','P','<','s','8','+','H','.','D','>','v','^','d']
compare_names_all = methods_of_interest_print 
for k in [2, 3, 4]:
    for outlier_style in ['multi_normal0', 'multi_normal1','gamma', 'block_normal6']:
        for dataset in ['ERO', 'BAO', 'RGGO']:
            for p in p_list_dict[dataset]:
                change_var_values = np.array(eta_list_dict[dataset][p])
                dataset_name_full = []
                mse_res_all_full = []
                final_upset_all_full = []
                change_var_values_list = []
                for eta in change_var_values:
                    try:
                        dataset_long, mse_res_all, final_upset_all, _ = extract_results(dataset=dataset, 
                            p=p, eta=eta, outlier_style=outlier_style, N=360, k=k)
                        dataset_name_full.append(dataset_long)
                        mse_res_all_full.append(mse_res_all)
                        final_upset_all_full.append(final_upset_all)
                        change_var_values_list.append(eta)
                    except FileNotFoundError:
                        print('No result yet for {}, p={}, eta={}.'.format(dataset,
                            p, eta))
                change_var_values = np.array(change_var_values_list)
                mse_res_all = np.array(mse_res_all_full)[:, 0, :, :, 0] # dim 0 is eta dim, dim 1 is method dim, dim 2 is 10 runs
                mse_res_mean = np.mean(mse_res_all, axis=2)
                mse_res_std = np.std(mse_res_all, axis=2)
                results_mean = mse_res_mean.transpose()
                results_std = mse_res_std.transpose()
                plt.figure(figsize=[8,6])
                plt.rcParams.update({'font.size': 12})
                plt.xticks(np.arange(change_var_values.min(), change_var_values.max()+0.1, step=0.1))
                for j in range(len(compare_names_all)-len(baselines)):
                    plt.errorbar(change_var_values, results_mean[j], yerr=results_std[j], label=compare_names_all[j],alpha=0.8, fmt=figure_markers[j%len(figure_markers)], ls='--')
                for j in range(len(compare_names_all)-len(baselines), len(compare_names_all)):
                    plt.errorbar(change_var_values, results_mean[j], yerr=results_std[j], label=compare_names_all[j],alpha=0.7, fmt=figure_markers[j%len(figure_markers)], ls='None')
                plt.xlabel(r'$\eta$',fontsize=22)
                plt.ylabel('MSE',fontsize=22, labelpad=-5)
                plt.rcParams.update({'font.size': 12})    
                title_name = 'k'+str(k)+dataset+'p'+str(int(100*p))+'N360style'+outlier_style
                plt.title(title_name)
                if save:
                    print('Saving figure!')
                    save_name = '../ksync_ablation_plots/k'+str(k)+dataset+'p'+str(int(100*p))+'N360style'+outlier_style+'.pdf'
                    plt.savefig(save_name,format='pdf')
                if dataset == 'ERO' and p == 0.05:
                    plt.legend(framealpha=0.0,fontsize=15,loc='lower left', ncol=4, bbox_to_anchor=(0, -0.4))
                plt.show()